# Byte Pair Encoding and DNA Sequence Classification with Deep Learning

This project utilizes Byte Pair Encoding (BPE) for tokenization and applies deep learning techniques to classify DNA sequences. It draws inspiration and knowledge from various sources, including Kaggle and scientific literature, to implement a solution that leverages SentencePiece for BPE and PyTorch for constructing a neural network model.

## Dependencies

- SentencePiece: For tokenization using Byte Pair Encoding.
- NumPy: For linear algebra operations.
- Pandas: For data processing and CSV file I/O.
- Scikit-learn: For feature extraction through CountVectorizer.
- PyTorch: For creating and training the neural network model.

## Dataset

The dataset comprises DNA sequences from humans, chimpanzees, and dogs. The aim is to classify these sequences into various categories based on their genetic information.

## Preprocessing

The preprocessing steps involve converting DNA sequences into k-mers, which are subsequences of length k. This representation is then used to transform the sequences into numerical format suitable for machine learning models.

## Execution Steps

1. Preprocess the data to generate k-mers.
2. Train the SentencePiece model for tokenization.
3. Load the tokenized data into a PyTorch DataLoader.
4. Define the neural network model, loss function, and optimizer.
5. Train the model using the training dataset.
6. Validate the model's performance on a test dataset.


In [ ]:
#https://www.kaggle.com/chuckzzzz/dna-classification-with-deep-learning-part-1/notebook
#https://www.kaggle.com/nageshsingh/demystify-dna-sequencing-with-machine-learning/notebook
#https://pypi.org/project/sentencepiece/
#http://ethen8181.github.io/machine-learning/deep_learning/subword/bpe.html

## Tokenization

Utilizes SentencePiece for Byte Pair Encoding, which is crucial for handling the vast vocabulary inherent in DNA sequences.

In [1]:
import sentencepiece as spm

model_name = 'genes'

In [ ]:
class Tokenizer:

    def __init__(self, filepath=model_name+'.model'):
        self.sp = spm.SentencePieceProcessor(model_file=filepath)

    def encode(self, text, t=int):
        return self.sp.encode(text, out_type=t)

    def decode(self, pieces):
        return self.sp.decode(pieces)

    @staticmethod
    def train(input_file='data/raw_sents.txt', model_prefix='sp_model', vocab_size=30522):
        spm.SentencePieceTrainer.train(input=input_file, model_prefix=model_prefix, vocab_size=vocab_size,
                                       #input_sentence_size=2 ** 16, shuffle_input_sentence=True)
                                       input_sentence_size=number_of_lines, shuffle_input_sentence=True)

In [ ]:
#train
Tokenizer.train(input_file='data.txt', model_prefix='python_tokenizer_30k', vocab_size=30000) #model_prefix is model storage name

In [ ]:
#instantiate tokenizer model
tokenizer = Tokenizer('python_tokenizer.model')

In [ ]:
#tokenize code (ie encode)
#with words
#tokens = tokenizer.encode(data[1][0],t=str)
#with numbers
tokens = tokenizer.encode(data[1][0])
decoded = tokenizer.decode(tokens)

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import random
import pickle
import matplotlib.pyplot as plt

# 0. Background
The basic background of the data used here is exceptionally covered in the kernel provided by the uploader for this dataset [1]. In this notebook, we will base the analysis on kmer-counting and expand on this strategy using various models.

https://en.wikipedia.org/wiki/K-mer

# 1. Preprocess¶
Most of the preprocessing done here is based on [1]. The main difference is how kmer is represented as a matrix. The original method used a 4-gramers for 6-mers. This should be equivalent to a 9-mer theoratically. For instance, ["ACCAT","CCACTA","CACTAA","ACTAAG"] is equivalent to "ACCACTAG" in terms of the information it contains. Therefore, the preprocessing here does kmer counting for all the train sequences. It then selects the top X number of most frequent kmer as feature to transform the each sequence. For example, if a sequence is "ACGTAGACGT" and the top most frequented kmer feature is ["ACGT","GTAG","GGCC"], the transfromed matrix for this sequence is [2,1,0].

Since human, chimpanzee and dog are all mammals, we will consider these DNA data as a single dataset. The train and validation split is done on this combined dataset as well. 80% of the sequences are used for training and the rest is used for test.

In [6]:
def getKmers(sequence, size=8):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

In [8]:
human = pd.read_table('human.txt')
chimp = pd.read_table('chimpanzee.txt')
dog = pd.read_table('dog.txt')


In [9]:
def generate_dataset(dfs,kmer_size,max_features,split=5):
    kmer_dfs=[]
    for cur_df in dfs:
        cur_df['words']=cur_df.apply(lambda x: getKmers(x['sequence'],size=kmer_size), axis=1)
        cur_df=cur_df.drop('sequence',axis=1)
        kmer_dfs.append(cur_df)
    all_data=pd.concat(kmer_dfs).reset_index(drop=True)
    perm=np.random.permutation(len(all_data)) #shuffle the data
    test_data=all_data[:len(all_data)//split]
    train_data=all_data[len(all_data)//split:]
    train_kmers=[]
    for cur_kmer_list in train_data.words.values:
        train_kmers.extend(cur_kmer_list)
    vectorizer = CountVectorizer(max_features=max_features).fit(train_kmers) 
    
    print(train_data["class"].value_counts())
    print(test_data["class"].value_counts())
    
    X_train=[]
    Y_train=[]
    X_test=[]
    Y_test=[]
    for cur_data, label in zip(train_data['words'],train_data['class']):
        cur_transformed=vectorizer.transform(cur_data)
        X_train.append(cur_transformed.toarray().sum(axis=0))
        Y_train.append(label)
    for cur_data, label in zip(test_data['words'],test_data['class']):
        cur_transformed=vectorizer.transform(cur_data)
        X_test.append(cur_transformed.toarray().sum(axis=0))
        Y_test.append(label)  
    return X_train, Y_train, X_test, Y_test
X_train, Y_train, X_test, Y_test=generate_dataset([human,chimp,dog],kmer_size=9,max_features=1500)

6    1702
4     910
3     772
0     753
1     551
2     476
5     342
Name: class, dtype: int64
6    422
1    243
3    223
4    197
0    143
2     81
5     67
Name: class, dtype: int64


# 2. Model
For this kernel, we will leverage the power of deep learning using Pytorch. The most simple model is a feedforward neural network composed of an input layer, several hidden layers and a final output layer. The number of nodes in input layer represents the number of features and the number of nodes in output layer represents number of classes. Here is a general schematic of what a feedforward neural network looks like:

FNN Schematic

The specific configuration of our experimental model has two hidden layers. We can fine tune model structure after we have shown the prototye works fine on our data (i.e. the model is learning).

In [10]:
import torch.utils.data as data_utils
import torch
from torch import nn
from torch.utils.data import DataLoader

In [11]:
class NN(nn.Module):
    def __init__(self, in_dim, n_hidden_1, n_hidden_2, out_dim):
        super(NN, self).__init__()
        self.in_dim=in_dim
        self.n_hidden_1=n_hidden_1
        self.n_hidden_2=n_hidden_2
        self.out_dim=out_dim
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True))
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True))
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2, out_dim),
            nn.ReLU(True))

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

# 2.1 Numpy to Tensor
This might be an over-simplification, but deep learning at its core is really just a lot of matrix operations. These matrices are termed tensor. Despite its different nomenclature, it does support most of the functions we can find in an numpy array. If interested, you may google the difference between numpy arrays and tensors. But basically, tensor is a multi-dimensional array that can operate on both cpu and gpu. When on GPU, the matrix operation will be significantly faster than numpy array on CPU as shwon in the diagram below thanks to this benchmark[3].

diagram of runtimeThis is why many deep learning projects use GPU to do training instead of CPU. But since this DNA sequence dataset isn't that much and our model is fairly simple, CPU works just fine. We will use some built-in functions of Pytorch to convert numpy arrays into tensors and dataloaders.

In [12]:
train_data = data_utils.TensorDataset(torch.tensor(X_train), torch.tensor(Y_train))
train_loader = data_utils.DataLoader(train_data, batch_size=4, shuffle=True)
test_data=data_utils.TensorDataset(torch.tensor(X_test), torch.tensor(Y_test))
test_loader=data_utils.DataLoader(test_data, batch_size=4, shuffle=True)
n_features=len(train_data[0][0])
n_labels=7

# 2.2 Hyperparameters¶
Deep learning also involves controlling parameters that are not part of the model itself. These parameters are termed hyperparameters as they control how the model learns from training data. For experimentation, we used emprically reasonable values for these parameters.

In [13]:
learning_rate=5e-3
num_epochs=10
torch.manual_seed(2021)
model = NN(n_features, n_features//5, n_features//10, n_labels)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
def train_model(model,num_epochs,train_loader,criterion,optimizer,verbose=False,learning_rate=5e-3):
    for epoch in range(num_epochs):
        if(verbose):
            print('*' * 10)
            print(f'epoch {epoch+1}')
        running_loss = 0.0
        running_acc = 0.0
        for i, data in enumerate(train_loader, 1):
            cur_seq, label = data
            cur_seq=cur_seq.float()
            label=label.squeeze_()
            optimizer.zero_grad()
            out = model(cur_seq)
            loss = criterion(out, label)
            running_loss += loss.item()
            _, pred = torch.max(out, 1)
            running_acc += (pred == label).float().mean()
            loss.backward()
            optimizer.step()
            if (verbose):
                if i % 400 == 0:
                    print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
        if(verbose):
            print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    return running_acc/i, model
def validate_model(model,test_loader):
        # eval
    correct = 0
    total = 0
    with torch.no_grad():
        for data in test_loader:
            cur_seq, labels = data
            cur_seq=cur_seq.float()
            labels=labels.squeeze_()
            # calculate outputs by running images through the network
            out = model(cur_seq)
            _, predicted = torch.max(out, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct/total

# 2.3 Model Sanity Test¶
It's usally good practice to try overfitting the model on the traning data first. If the model can overfit the data, that means this dataset is learnable by our naive implementation of neural network. Clearly, overfitting will hurt the model performance on unseen data as shown by the diagram below. However, avoiding overfitting would be a later job once we confirm the model is learning from the dataset. Red is the error on test dataset while blue is the error on training dataset.

In [15]:
acc,model=train_model(model,20,train_loader,criterion,optimizer,verbose=False,learning_rate=5e-3)
print("Model training accuracy is %.4f"%acc)

Model training accuracy is 0.9898


In [16]:
validate_model(model,test_loader)

0.8023255813953488

In [19]:
learning_rate=5e-3
num_epochs=25
torch.manual_seed(2021)
model = NN(n_features, n_features//5, n_features//10, n_labels)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [20]:
acc,model=train_model(model,num_epochs,train_loader,criterion,optimizer,verbose=True,learning_rate=5e-3)
validate_model(model,test_loader)

**********
epoch 1
[1/25] Loss: 1.899268, Acc: 0.306250
[1/25] Loss: 1.875588, Acc: 0.301250
[1/25] Loss: 1.852687, Acc: 0.305208
Finish 1 epoch, Loss: 1.844667, Acc: 0.306463
**********
epoch 2
[2/25] Loss: 1.759007, Acc: 0.308125
[2/25] Loss: 1.742570, Acc: 0.311875
[2/25] Loss: 1.712514, Acc: 0.328542
Finish 2 epoch, Loss: 1.701493, Acc: 0.332062
**********
epoch 3
[3/25] Loss: 1.579501, Acc: 0.411875
[3/25] Loss: 1.522568, Acc: 0.434687
[3/25] Loss: 1.492915, Acc: 0.455208
Finish 3 epoch, Loss: 1.476195, Acc: 0.468773
**********
epoch 4
[4/25] Loss: 1.288463, Acc: 0.588125
[4/25] Loss: 1.254246, Acc: 0.608125
[4/25] Loss: 1.205462, Acc: 0.626250
Finish 4 epoch, Loss: 1.187723, Acc: 0.633624
**********
epoch 5
[5/25] Loss: 0.992845, Acc: 0.700000
[5/25] Loss: 0.981655, Acc: 0.709687
[5/25] Loss: 0.933177, Acc: 0.718958
Finish 5 epoch, Loss: 0.914140, Acc: 0.725853
**********
epoch 6
[6/25] Loss: 0.705540, Acc: 0.785000
[6/25] Loss: 0.668124, Acc: 0.806875
[6/25] Loss: 0.664383, Acc:

0.8088662790697675

# References

https://www.kaggle.com/nageshsingh/demystify-dna-sequencing-with-machine-learning#Demystify-DNA-Sequencing-with-Machine-Learning


https://upload.wikimedia.org/wikipedia/commons/c/c2/MultiLayerNeuralNetworkBigger_english.png


https://medium.com/thenoobengineer/numpy-arrays-vs-tensors-c58ea54f0e59


https://upload.wikimedia.org/wikipedia/commons/f/fc/Overfitting.png


https://github.com/L1aoXingyu/pytorch-beginner/tree/master/03-Neural%20Network